In [22]:
!pip install ortools


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\vjame\\anaconda3\\Lib\\site-packages\\google\\protobuf\\~nternal\\_api_implementation.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0


In [2]:
pip install --upgrade ortools


In [3]:
import ortools
print(ortools.__version__)


9.6.2534


In [8]:
#!/usr/bin/env python3
# This Python file uses the following encoding: utf-8
# Copyright 2015 Tin Arm Engineering AB
# Copyright 2018 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Capacitated Vehicle Routing Problem (CVRP).

   This is a sample using the routing library python wrapper to solve a CVRP
   problem while allowing multiple trips, i.e., vehicles can return to a depot
   to reset their load ("reload").

   A description of the CVRP problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.

   In order to implement multiple trips, new nodes are introduced at the same
   locations of the original depots. These additional nodes can be dropped
   from the schedule at 0 cost.

   The max_slack parameter associated to the capacity constraints of all nodes
   can be set to be the maximum of the vehicles' capacities, rather than 0 like
   in a traditional CVRP. Slack is required since before a solution is found,
   it is not known how much capacity will be transferred at the new nodes. For
   all the other (original) nodes, the slack is then re-set to 0.

   The above two considerations are implemented in `add_capacity_constraints()`.

   Last, it is useful to set a large distance between the initial depot and the
   new nodes introduced, to avoid schedules having spurious transits through
   those new nodes unless it's necessary to reload. This consideration is taken
   into account in `create_distance_evaluator()`.
"""

from functools import partial

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


###########################
# Problem Data Definition #
###########################
def create_data_model():
    """Stores the data for the problem"""
    data = {}
    _capacity = 15
    # Locations in block unit
    _locations = [
        (4, 4),  # depot
        (4, 4),  # unload depot_first
        (4, 4),  # unload depot_second
        (4, 4),  # unload depot_third
        (4, 4),  # unload depot_fourth
        (4, 4),  # unload depot_fifth
        (2, 0),
        (8, 0),  # locations to visit
        (0, 1),
        (1, 1),
        (5, 2),
        (7, 2),
        (3, 3),
        (6, 3),
        (5, 5),
        (8, 5),
        (1, 6),
        (2, 6),
        (3, 7),
        (6, 7),
        (0, 8),
        (7, 8)
    ]
    # Compute locations in meters using the block dimension defined as follow
    # Manhattan average block: 750ft x 264ft -> 228m x 80m
    # here we use: 114m x 80m city block
    # src: https://nyti.ms/2GDoRIe 'NY Times: Know Your distance'
    data['locations'] = [(l[0] * 114, l[1] * 80) for l in _locations]
    data['num_locations'] = len(data['locations'])
    data['demands'] = \
          [0, # depot
           -_capacity, # unload depot_first
           -_capacity, # unload depot_second
           -_capacity, # unload depot_third
           -_capacity, # unload depot_fourth
           -_capacity, # unload depot_fifth
           3, 3, # 1, 2
           3, 4, # 3, 4
           3, 4, # 5, 6
           8, 8, # 7, 8
           3, 3, # 9,10
           3, 3, # 11,12
           4, 4, # 13, 14
           8, 8] # 15, 16
    data['time_per_demand_unit'] = 5  # 5 minutes/unit
    data['num_vehicles'] = 3
    data['vehicle_capacity'] = _capacity
    data['vehicle_max_distance'] = 10_000
    data['vehicle_max_time'] = 1_500
    data[
        'vehicle_speed'] = 5 * 60 / 3.6  # Travel speed: 5km/h to convert in m/min
    data['depot'] = 0
    return data


#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
    return (abs(position_1[0] - position_2[0]) +
            abs(position_1[1] - position_2[1]))


def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in range(data['num_locations']):
        _distances[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            # Forbid start/end/reload node to be consecutive.
            elif from_node in range(6) and to_node in range(6):
                _distances[from_node][to_node] = data['vehicle_max_distance']
            else:
                _distances[from_node][to_node] = (manhattan_distance(
                    data['locations'][from_node], data['locations'][to_node]))

    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return distance_evaluator


def add_distance_dimension(routing, manager, data, distance_evaluator_index):
    """Add Global Span constraint"""
    del manager
    distance = 'Distance'
    routing.AddDimension(
        distance_evaluator_index,
        0,  # null slack
        data['vehicle_max_distance'],  # maximum distance per vehicle
        True,  # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)


def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, from_node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(from_node)]

    return demand_evaluator


def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
    """Adds capacity constraint"""
    vehicle_capacity = data['vehicle_capacity']
    capacity = 'Capacity'
    routing.AddDimension(
        demand_evaluator_index,
        vehicle_capacity,
        vehicle_capacity,
        True,  # start cumul to zero
        capacity)

    # Add Slack for reseting to zero unload depot nodes.
    # e.g. vehicle with load 10/15 arrives at node 1 (depot unload)
    # so we have CumulVar = 10(current load) + -15(unload) + 5(slack) = 0.
    capacity_dimension = routing.GetDimensionOrDie(capacity)
    # Allow to drop reloading nodes with zero cost.
    for node in [1, 2, 3, 4, 5]:
        node_index = manager.NodeToIndex(node)
        routing.AddDisjunction([node_index], 0)

    # Allow to drop regular node with a cost.
    for node in range(6, len(data['demands'])):
        node_index = manager.NodeToIndex(node)
        capacity_dimension.SlackVar(node_index).SetValue(0)
        routing.AddDisjunction([node_index], 100_000)


###########
# Printer #
###########
def print_solution(data, manager, routing, assignment):  # pylint:disable=too-many-locals
    """Prints assignment on console"""
    print(f'Objective: {assignment.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    dropped = []
    for order in range(6, routing.nodes()):
        index = manager.NodeToIndex(order)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(order)
    print(f'dropped orders: {dropped}')
    for reload in range(1, 6):
        index = manager.NodeToIndex(reload)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(reload)
    print(f'dropped reload stations: {dropped}')

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        distance = 0
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            plan_output += (
                f' {manager.IndexToNode(index)} '
                f'Load({assignment.Min(load_var)}) ->'
            )
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            distance += routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id)
        load_var = capacity_dimension.CumulVar(index)
        plan_output += (
            f' {manager.IndexToNode(index)} '
            f'Load({assignment.Min(load_var)})\n')
        plan_output += f'Distance of the route: {distance}m\n'
        plan_output += f'Load of the route: {assignment.Min(load_var)}\n'
        print(plan_output)
        total_distance += distance
        total_load += assignment.Min(load_var)
    print(f'Total Distance of all routes: {total_distance}m')
    print(f'Total Load of all routes: {total_load}')
    print(f'Total Time of all routes: {total_time}min')


########
# Main #
########
def main():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define weight of each edge
    distance_evaluator_index = routing.RegisterTransitCallback(
        partial(create_distance_evaluator(data), manager))
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator_index)

    # Add Distance constraint to minimize the longuest route
    add_distance_dimension(routing, manager, data, distance_evaluator_index)

    # Add Capacity constraint
    demand_evaluator_index = routing.RegisterUnaryTransitCallback(
        partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, manager, data, demand_evaluator_index)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)  # pylint: disable=no-member
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(3)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == '__main__':
    main()


Objective: 295452
dropped orders: []
dropped reload stations: [3]
Route for vehicle 0:
 0 Load(0) -> 6 Load(0) -> 9 Load(3) -> 8 Load(7) -> 4 Load(10) -> 14 Load(0) -> 21 Load(3) -> 19 Load(11) -> 0 Load(15)
Distance of the route: 2876m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) -> 18 Load(0) -> 1 Load(4) -> 20 Load(0) -> 16 Load(8) -> 17 Load(11) -> 0 Load(14)
Distance of the route: 2260m
Load of the route: 14

Route for vehicle 2:
 0 Load(0) -> 10 Load(0) -> 11 Load(3) -> 7 Load(7) -> 15 Load(10) -> 2 Load(13) -> 12 Load(0) -> 5 Load(8) -> 13 Load(0) -> 0 Load(8)
Distance of the route: 2716m
Load of the route: 8

Total Distance of all routes: 7852m
Total Load of all routes: 37
Total Time of all routes: 0min


In [9]:
#!/usr/bin/env python3
# This Python file uses the following encoding: utf-8
# Copyright 2015 Tin Arm Engineering AB
# Copyright 2018 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Capacitated Vehicle Routing Problem (CVRP).

   This is a sample using the routing library python wrapper to solve a CVRP
   problem while allowing multiple trips, i.e., vehicles can return to a depot
   to reset their load ("reload").

   A description of the CVRP problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.

   In order to implement multiple trips, new nodes are introduced at the same
   locations of the original depots. These additional nodes can be dropped
   from the schedule at 0 cost.

   The max_slack parameter associated to the capacity constraints of all nodes
   can be set to be the maximum of the vehicles' capacities, rather than 0 like
   in a traditional CVRP. Slack is required since before a solution is found,
   it is not known how much capacity will be transferred at the new nodes. For
   all the other (original) nodes, the slack is then re-set to 0.

   The above two considerations are implemented in `add_capacity_constraints()`.

   Last, it is useful to set a large distance between the initial depot and the
   new nodes introduced, to avoid schedules having spurious transits through
   those new nodes unless it's necessary to reload. This consideration is taken
   into account in `create_distance_evaluator()`.
"""

from functools import partial

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


###########################
# Problem Data Definition #
###########################
def create_data_model():
    """Stores the data for the problem"""
    data = {}
    # Locations in block unit
    _locations = [
        (4, 4),  # depot
        (4, 4),  # unload depot_first
        (4, 4),  # unload depot_second
        (4, 4),  # unload depot_third
        (4, 4),  # unload depot_fourth
        (4, 4),  # unload depot_fifth
        (2, 0),
        (8, 0),  # locations to visit
        (0, 1),
        (1, 1),
        (5, 2),
        (7, 2),
        (3, 3),
        (6, 3),
        (5, 5),
        (8, 5),
        (1, 6),
        (2, 6),
        (3, 7),
        (6, 7),
        (0, 8),
        (7, 8)
    ]
    # Compute locations in meters using the block dimension defined as follow
    # Manhattan average block: 750ft x 264ft -> 228m x 80m
    # here we use: 114m x 80m city block
    # src: https://nyti.ms/2GDoRIe 'NY Times: Know Your distance'
    data['locations'] = [(l[0] * 114, l[1] * 80) for l in _locations]
    data['num_locations'] = len(data['locations'])
    data['demands'] = \
          [0, # depot
           -8, # unload depot_first
           -8, # unload depot_second
           -12, # unload depot_third
           -12, # unload depot_fourth
           -13, # unload depot_fifth
           3, 3, # 1, 2
           3, 4, # 3, 4
           3, 4, # 5, 6
           8, 8, # 7, 8
           3, 3, # 9,10
           3, 3, # 11,12
           4, 4, # 13, 14
           8, 8] # 15, 16
    data['time_per_demand_unit'] = 5  # 5 minutes/unit
    data['num_vehicles'] = 3
    data['vehicle_capacity'] = [8, 12, 13]
    data['vehicle_max_distance'] = 10_000
    data['vehicle_max_time'] = 1_500
    data['vehicle_speed'] = 5 * 60 / 3.6  # Travel speed: 5km/h to convert in m/min
    data['depot'] = 0
    return data


#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
    return (abs(position_1[0] - position_2[0]) +
            abs(position_1[1] - position_2[1]))


def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in range(data['num_locations']):
        _distances[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            # Forbid start/end/reload node to be consecutive.
            elif from_node in range(6) and to_node in range(6):
                _distances[from_node][to_node] = data['vehicle_max_distance']
            else:
                _distances[from_node][to_node] = (manhattan_distance(
                    data['locations'][from_node], data['locations'][to_node]))

    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return distance_evaluator


def add_distance_dimension(routing, manager, data, distance_evaluator_index):
    """Add Global Span constraint"""
    del manager
    distance = 'Distance'
    routing.AddDimension(
        distance_evaluator_index,
        0,  # null slack
        data['vehicle_max_distance'],  # maximum distance per vehicle
        True,  # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)


def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, from_node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(from_node)]

    return demand_evaluator


def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
    """Adds capacity constraint"""
    vehicle_capacities = data['vehicle_capacity']
    capacity = 'Capacity'
    routing.AddDimensionWithVehicleCapacity(
        demand_evaluator_index,
        0,  # null slack
        vehicle_capacities,
        True,  # start cumul to zero
        capacity)

    capacity_dimension = routing.GetDimensionOrDie(capacity)
    # Allow to drop reloading nodes with zero cost.
    for node in [1, 2, 3, 4, 5]:
        node_index = manager.NodeToIndex(node)
        routing.AddDisjunction([node_index], 0)

    # Allow to drop regular node with a cost.
    for node in range(6, len(data['demands'])):
        node_index = manager.NodeToIndex(node)
        capacity_dimension.SlackVar(node_index).SetValue(0)
        routing.AddDisjunction([node_index], 100_000)


###########
# Printer #
###########
def print_solution(data, manager, routing, assignment):  # pylint:disable=too-many-locals
    """Prints assignment on console"""
    print(f'Objective: {assignment.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    dropped = []
    for order in range(6, routing.nodes()):
        index = manager.NodeToIndex(order)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(order)
    print(f'dropped orders: {dropped}')
    for reload in range(1, 6):
        index = manager.NodeToIndex(reload)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(reload)
    print(f'dropped reload stations: {dropped}')

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        distance = 0
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            plan_output += (
                f' {manager.IndexToNode(index)} '
                f'Load({assignment.Min(load_var)}) ->'
            )
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            distance += routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id)
        load_var = capacity_dimension.CumulVar(index)
        plan_output += (
            f' {manager.IndexToNode(index)} '
            f'Load({assignment.Min(load_var)})\n')
        plan_output += f'Distance of the route: {distance}m\n'
        plan_output += f'Load of the route: {assignment.Min(load_var)}\n'
        print(plan_output)
        total_distance += distance
        total_load += assignment.Min(load_var)
    print(f'Total Distance of all routes: {total_distance}m')
    print(f'Total Load of all routes: {total_load}')


########
# Main #
########
def main():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define weight of each edge
    distance_evaluator_index = routing.RegisterTransitCallback(
        partial(create_distance_evaluator(data), manager))
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator_index)

    # Add Distance constraint to minimize the longuest route
    add_distance_dimension(routing, manager, data, distance_evaluator_index)

    # Add Capacity constraint
    demand_evaluator_index = routing.RegisterUnaryTransitCallback(
        partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, manager, data, demand_evaluator_index)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)  # pylint: disable=no-member
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(3)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == '__main__':
    main()


Objective: 477968
dropped orders: [20, 21]
dropped reload stations: [20, 21, 3, 4]
Route for vehicle 0:
 0 Load(0) -> 12 Load(0) -> 2 Load(8) -> 14 Load(0) -> 19 Load(3) -> 0 Load(7)
Distance of the route: 1324m
Load of the route: 7

Route for vehicle 1:
 0 Load(0) -> 6 Load(0) -> 9 Load(3) -> 8 Load(7) -> 1 Load(10) -> 16 Load(2) -> 17 Load(5) -> 18 Load(8) -> 0 Load(12)
Distance of the route: 2716m
Load of the route: 12

Route for vehicle 2:
 0 Load(0) -> 15 Load(0) -> 7 Load(3) -> 11 Load(6) -> 10 Load(10) -> 5 Load(13) -> 13 Load(0) -> 0 Load(8)
Distance of the route: 2328m
Load of the route: 8

Total Distance of all routes: 6368m
Total Load of all routes: 27


In [12]:
#!/usr/bin/env python3
# This Python file uses the following encoding: utf-8
# Copyright 2015 Tin Arm Engineering AB
# Copyright 2018 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Capacitated Vehicle Routing Problem (CVRP).

   This is a sample using the routing library python wrapper to solve a CVRP
   problem while allowing multiple trips, i.e., vehicles can return to a depot
   to reset their load ("reload").

   A description of the CVRP problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.

   In order to implement multiple trips, new nodes are introduced at the same
   locations of the original depots. These additional nodes can be dropped
   from the schedule at 0 cost.

   The max_slack parameter associated to the capacity constraints of all nodes
   can be set to be the maximum of the vehicles' capacities, rather than 0 like
   in a traditional CVRP. Slack is required since before a solution is found,
   it is not known how much capacity will be transferred at the new nodes. For
   all the other (original) nodes, the slack is then re-set to 0.

   The above two considerations are implemented in `add_capacity_constraints()`.

   Last, it is useful to set a large distance between the initial depot and the
   new nodes introduced, to avoid schedules having spurious transits through
   those new nodes unless it's necessary to reload. This consideration is taken
   into account in `create_distance_evaluator()`.
"""

from functools import partial

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


###########################
# Problem Data Definition #
###########################
def create_data_model():
    """Stores the data for the problem"""
    data = {}
    # Locations in block unit
    _locations = [
        (4, 4),  # depot
        (4, 4),  # unload depot_first
        (4, 4),  # unload depot_second
        (4, 4),  # unload depot_third
        (4, 4),  # unload depot_fourth
        (4, 4),  # unload depot_fifth
        (2, 0),
        (8, 0),  # locations to visit
        (0, 1),
        (1, 1),
        (5, 2),
        (7, 2),
        (3, 3),
        (6, 3),
        (5, 5),
        (8, 5),
        (1, 6),
        (2, 6),
        (3, 7),
        (6, 7),
        (0, 8),
        (7, 8)
    ]
    # Compute locations in meters using the block dimension defined as follow
    # Manhattan average block: 750ft x 264ft -> 228m x 80m
    # here we use: 114m x 80m city block
    # src: https://nyti.ms/2GDoRIe 'NY Times: Know Your distance'
    data['locations'] = [(l[0] * 114, l[1] * 80) for l in _locations]
    data['num_locations'] = len(data['locations'])
    data['demands'] = \
          [0, # depot
           -8, # unload depot_first
           -8, # unload depot_second
           -12, # unload depot_third
           -12, # unload depot_fourth
           -13, # unload depot_fifth
           3, 3, # 1, 2
           3, 4, # 3, 4
           3, 4, # 5, 6
           8, 8, # 7, 8
           3, 3, # 9,10
           3, 3, # 11,12
           4, 4, # 13, 14
           8, 8] # 15, 16
    data['time_per_demand_unit'] = 5  # 5 minutes/unit
    data['num_vehicles'] = 3
    data['vehicle_capacity'] = [8, 12, 13]
    data['vehicle_max_distance'] = 10_000
    data['vehicle_max_time'] = 1_500
    data['vehicle_speed'] = 5 * 60 / 3.6  # Travel speed: 5km/h to convert in m/min
    data['depot'] = 0
    return data


#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
    return (abs(position_1[0] - position_2[0]) +
            abs(position_1[1] - position_2[1]))


def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in range(data['num_locations']):
        _distances[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            # Forbid start/end/reload node to be consecutive.
            elif from_node in range(6) and to_node in range(6):
                _distances[from_node][to_node] = data['vehicle_max_distance']
            else:
                _distances[from_node][to_node] = (manhattan_distance(
                    data['locations'][from_node], data['locations'][to_node]))

    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return distance_evaluator


def add_distance_dimension(routing, manager, data, distance_evaluator_index):
    """Add Global Span constraint"""
    del manager
    distance = 'Distance'
    routing.AddDimension(
        distance_evaluator_index,
        0,  # null slack
        data['vehicle_max_distance'],  # maximum distance per vehicle
        True,  # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)


def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, from_node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(from_node)]

    return demand_evaluator


def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
    """Adds capacity constraint"""
    vehicle_capacities = data['vehicle_capacity']
    capacity = 'Capacity'
    routing.AddDimensionWithVehicleCapacity(
        demand_evaluator_index,
        0,  # null slack
        vehicle_capacities,
        True,  # start cumul to zero
        capacity)

    capacity_dimension = routing.GetDimensionOrDie(capacity)
    # Allow to drop reloading nodes with zero cost.
    for node in [1, 2, 3, 4, 5]:
        node_index = manager.NodeToIndex(node)
        routing.AddDisjunction([node_index], 0)

    # Allow to drop regular node with a cost.
    for node in range(6, len(data['demands'])):
        node_index = manager.NodeToIndex(node)
        capacity_dimension.SlackVar(node_index).SetValue(0)
        routing.AddDisjunction([node_index], 100_000)


###########
# Printer #
###########
def print_solution(data, manager, routing, assignment):  # pylint:disable=too-many-locals
    """Prints assignment on console"""
    print(f'Objective: {assignment.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    dropped = []
    for order in range(6, routing.nodes()):
        index = manager.NodeToIndex(order)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(order)
    print(f'dropped orders: {dropped}')
    for reload in range(1, 6):
        index = manager.NodeToIndex(reload)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(reload)
    print(f'dropped reload stations: {dropped}')

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        distance = 0
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            plan_output += (
                f' {manager.IndexToNode(index)} '
                f'Load({assignment.Min(load_var)}) ->'
            )
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            distance += routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id)
        load_var = capacity_dimension.CumulVar(index)
        plan_output += (
            f' {manager.IndexToNode(index)} '
            f'Load({assignment.Min(load_var)})\n')
        plan_output += f'Distance of the route: {distance}m\n'
        plan_output += f'Load of the route: {assignment.Min(load_var)}\n'
        print(plan_output)
        total_distance += distance
        total_load += assignment.Min(load_var)
    print(f'Total Distance of all routes: {total_distance}m')
    print(f'Total Load of all routes: {total_load}')


########
# Main #
########
def main():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define weight of each edge
    distance_evaluator_index = routing.RegisterTransitCallback(
        partial(create_distance_evaluator(data), manager))
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator_index)

    # Add Distance constraint to minimize the longuest route
    add_distance_dimension(routing, manager, data, distance_evaluator_index)

    # Add Capacity constraint
    demand_evaluator_index = routing.RegisterUnaryTransitCallback(
        partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, manager, data, demand_evaluator_index)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)  # pylint: disable=no-member
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(3)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == '__main__':
    main()


Objective: 477968
dropped orders: [20, 21]
dropped reload stations: [20, 21, 3, 4]
Route for vehicle 0:
 0 Load(0) -> 12 Load(0) -> 2 Load(8) -> 14 Load(0) -> 19 Load(3) -> 0 Load(7)
Distance of the route: 1324m
Load of the route: 7

Route for vehicle 1:
 0 Load(0) -> 6 Load(0) -> 9 Load(3) -> 8 Load(7) -> 1 Load(10) -> 16 Load(2) -> 17 Load(5) -> 18 Load(8) -> 0 Load(12)
Distance of the route: 2716m
Load of the route: 12

Route for vehicle 2:
 0 Load(0) -> 15 Load(0) -> 7 Load(3) -> 11 Load(6) -> 10 Load(10) -> 5 Load(13) -> 13 Load(0) -> 0 Load(8)
Distance of the route: 2328m
Load of the route: 8

Total Distance of all routes: 6368m
Total Load of all routes: 27


In [3]:
import pandas as pd
pd.read_csv("../data/final_num_171.csv")
distance_matrix=pd.read_csv("../data/final_distance.csv")

c:\Users\vjame\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\vjame\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [5]:

matrix = distance_matrix.values
matrix[[0, 171]] = matrix[[171, 0]]

# 열 교체
matrix[:, [0, 171]] = matrix[:, [171, 0]]
lst=matrix
demands=[0,612, 306, 921, 982, 231, 1068, 173, 151, 35, 612, 122, 122, 140, 414, 45, 182, 224, 122, 306, 45, 45, 136, 224, 133, 60, 140, 61, 538, 136, 758, 224, 196, 61, 45, 820, 97, 35, 61, 285, 97, 224, 232, 232, 98, 98, 52, 605, 97, 133, 140, 140, 140, 97, 136, 97, 136, 136, 97, 133, 98, 61, 215, 125, 136, 136, 94, 94, 136, 280, 94, 97, 173, 1465, 35, 285, 384, 217, 332, 258, 984, 112, 73, 73, 151, 332, 1043, 440, 440, 196, 258, 760, 136, 332, 112, 378, 196, 52, 112, 1010, 151, 71, 950, 160, 378, 432, 280, 605, 210, 35, 35, 35, 125, 217, 173, 71, 322, 322, 436, 479, 1065, 387, 173, 322, 322, 71, 564, 423, 432, 215, 215, 92, 52, 160, 52, 92, 92, 160, 160, 280, 397, 35, 35, 35, 35, 35, 35, 35, 35, 35, 1231, 61, 61, 60, 35, 35, 35, 35, 35, 35, 60, 61, 94, 35, 60, 61, 182, 60, 136, 136, 414, 94]

In [30]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = lst
    data["demands"] = demands
    data["vehicle_capacities"] = [6100, 5000,3700 , 3000,1500]*2+[3000]
    data["num_vehicles"] = 11
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    #print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
    #    print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    #print(f"Total load of all routes: {total_load}")
    return solution.ObjectiveValue(),total_distance,total_load

def main(first,local_search):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        first
        #routing_enums_pb2.FirstSolutionStrategy.SAVINGS
    )
    search_parameters.local_search_metaheuristic = (
        local_search
        #routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(10)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        result.append(print_solution(data, manager, routing, solution))
result=[]
if __name__ == "__main__":
    for i in [routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC,routing_enums_pb2.FirstSolutionStrategy.DESCRIPTOR,routing_enums_pb2.FirstSolutionStrategy.SAVINGS,routing_enums_pb2.FirstSolutionStrategy.SWEEP,routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,routing_enums_pb2.FirstSolutionStrategy.ALL_UNPERFORMED,routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION,routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC,routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC,routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,routing_enums_pb2.FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE]:
        for j in [routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT,routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH,routing_enums_pb2.LocalSearchMetaheuristic.GENERIC_TABU_SEARCH]:
            main(i,j)

Total distance of all routes: 62870m
Total distance of all routes: 60906m
Total distance of all routes: 62870m
Total distance of all routes: 61343m
Total distance of all routes: 62870m
Total distance of all routes: 62870m
Total distance of all routes: 60906m
Total distance of all routes: 62870m
Total distance of all routes: 61343m
Total distance of all routes: 62870m


TypeError: 'google._upb._message.Descriptor' object cannot be interpreted as an integer

In [28]:
print("Solver status: ", solver.status())


NameError: name 'solver' is not defined

In [27]:
from ortools.constraint_solver import routing_enums_pb2

# FirstSolutionStrategy 열거형에 포함된 모든 전략을 리스트로 생성
first_solution_strategies = [
    name for name in dir(routing_enums_pb2.FirstSolutionStrategy)
    if not name.startswith('_') and name.isupper()
]

# 리스트 출력
print(first_solution_strategies)


['DESCRIPTOR', 'PATH_CHEAPEST_ARC', 'SAVINGS']


In [ ]:
for i in 

In [24]:
sum(demands)-sum([6100, 5000,3700 , 3000,1500]*2)

1837

In [13]:
from ortools.constraint_solver import routing_enums_pb2

# 모든 첫 번째 솔루션 전략 출력
print("Available First Solution Strategies:")
for strategy in dir(routing_enums_pb2.FirstSolutionStrategy):
    if not strategy.startswith('_'):
        print(strategy)


Available First Solution Strategies:
ByteSize
Clear
ClearExtension
ClearField
CopyFrom
DESCRIPTOR
DiscardUnknownFields
Extensions
FindInitializationErrors
FromString
HasExtension
HasField
IsInitialized
ListFields
MergeFrom
MergeFromString
PATH_CHEAPEST_ARC
ParseFromString
SerializePartialToString
SerializeToString
SetInParent
UnknownFields
WhichOneof
